# Product Multimodal Search using Amazon Nova Embedding and S3 Vectors
Demonstrate multimodal product search using text descriptions and product images from Amazon Berkeley Objects dataset.

- Amazon Nova Mutlimodal Embedding Model
- S3 Vectors

![Nova MM Embedding](./images/visual-product-search.png)

In [ ]:
!pip install boto3 pandas --upgrade
!pip install s3fs

In [ ]:
import boto3
import pandas as pd
import json
from datetime import datetime

model_id = 'amazon.nova-2-multimodal-embeddings-v1:0'
account_id = '<Your AWS account ID>'
dim = 3072
s3vector_bucket = "<Your s3 vectors bucket name>"
s3vector_index = "product"

bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
s3vectors = boto3.client("s3vectors", region_name="us-east-1")

## 1. Create S3 Vector Index

In [ ]:
# Create a S3 vector bucket if not exists
s3vectors = boto3.client("s3vectors", region_name="us-east-1")
try:
    s3vectors.create_vector_bucket(vectorBucketName=s3vector_bucket)
    print(f"Vector bucket '{s3vector_bucket}' created successfully.")
except Exception as ex:    
    print(f'Failed to create S3 vector bucket: {s3vector_bucket}', ex)

# Delete index
s3vectors.delete_index(
        vectorBucketName=s3vector_bucket,
        indexName=s3vector_index)

# Create an index in the vector store if not exists
try:
    s3vectors.create_index(
        vectorBucketName=s3vector_bucket,
        indexName=s3vector_index,
        dataType='float32',  # Common data type for vector embeddings
        dimension=dim,
        distanceMetric='cosine' # or 'euclidean'
    )
    print(f"Vector index '{s3vector_index}' created successfully in bucket '{s3vector_bucket}'.")
except Exception as ex:    
    print(f'Failed to create S3 vector index {s3vector_index} in bucket: {s3vector_bucket}', ex)

## 2. Load Amazon Berkeley Objects Dataset

In [ ]:
# Load product metadata
meta = pd.read_json("s3://amazon-berkeley-objects/listings/metadata/listings_0.json.gz", lines=True)

def func_(x):
    us_texts = [item["value"] for item in x if item["language_tag"] == "en_US"]
    return us_texts[0] if us_texts else None

meta = meta.assign(item_name_in_en_us=meta.item_name.apply(func_))
meta = meta[~meta.item_name_in_en_us.isna()][["item_id", "item_name_in_en_us", "main_image_id"]]
print(f"#products with US English title: {len(meta)}")
meta.head()

In [ ]:
# Load image metadata and merge
image_meta = pd.read_csv("s3://amazon-berkeley-objects/images/metadata/images.csv.gz")
dataset = meta.merge(image_meta, left_on="main_image_id", right_on="image_id")

# Create full S3 path for images
dataset = dataset.assign(img_full_path='s3://amazon-berkeley-objects/images/small/' + dataset.path.astype(str))
print(f"Final dataset size: {len(dataset)}")
dataset.head()

## 3. Generate Embeddings for Products

In [ ]:
def generate_text_embedding(text, purpose="GENERIC_INDEX"):
    """Generate embedding for text"""
    request_body = {
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingDimension": dim,
            "embeddingPurpose": purpose,
            "text": {"truncationMode": "END", "value": text}
        }
    }
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    result = json.loads(response['body'].read())
    return result["embeddings"][0]["embedding"]

import base64

def load_file_as_base64(s3_uri):
    """Download file from S3 and convert to base64"""
    bucket = s3_uri.split('/')[2]
    key = '/'.join(s3_uri.split('/')[3:])
    
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    return base64.b64encode(obj['Body'].read()).decode('utf-8')

def generate_image_embedding(s3_uri, purpose="GENERIC_INDEX"):
    """Generate embedding for image"""
    request_body = {
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingPurpose": purpose,
            "embeddingDimension": dim,
            "image": {
                "format": "jpeg",
                "detailLevel": "STANDARD_IMAGE",
                "source": {"bytes": load_file_as_base64(s3_uri)},
            },
        },
    }
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    result = json.loads(response['body'].read())
    return result["embeddings"][0]["embedding"]

In [ ]:
# Process products and generate embeddings
import numpy as np
from tqdm import tqdm

# Take first 100 products for demo
sample_dataset = dataset.head(100)
embeddings = []
batch_size = 10

for idx, row in tqdm(sample_dataset.iterrows(), total=len(sample_dataset), desc="Processing products"):
    try:
        # Generate text embedding
        text_embed = generate_text_embedding(row['item_name_in_en_us'])
        
        # Generate image embedding
        img_embed = generate_image_embedding(row['img_full_path'])
        
        # Combine embeddings (average)
        combined_embed = [(t + i) / 2 for t, i in zip(text_embed, img_embed)]
        
        # Store embeddings
        embeddings.extend([
            {
                "key": f"combined-{row['item_id']}",
                "data": {"float32": combined_embed},
                "metadata": {
                    "item_id": row['item_id'],
                    "item_name": row['item_name_in_en_us'],
                    "img_path": row['img_full_path'],
                    "type": "combined"
                }
            }
        ])
        
        # Batch upload to S3 vectors
        if len(embeddings) >= batch_size * 3:
            s3vectors.put_vectors(
                vectorBucketName=s3vector_bucket,
                indexName=s3vector_index,
                vectors=embeddings
            )
            print(f"Uploaded batch ending with item {row['item_id']}")
            embeddings = []
            
    except Exception as e:
        print(f"Error processing item {row['item_id']}: {e}")
        continue

# Upload remaining embeddings
if embeddings:
    s3vectors.put_vectors(
        vectorBucketName=s3vector_bucket,
        indexName=s3vector_index,
        vectors=embeddings
    )
    print("Uploaded final batch")

print("Embedding generation and upload complete")

## 4. Test Search Functionality

In [ ]:
def search_products(query_embed, topK=3):
    """Search products using embedding"""
    response = s3vectors.query_vectors(
        vectorBucketName=s3vector_bucket,
        indexName=s3vector_index,
        queryVector={"float32": query_embed},
        topK=topK,
        returnDistance=True,
        returnMetadata=True
    )
    return response

def display_results(search_response):
    """Display search results with images"""
    from IPython.display import display, HTML
    
    html_content = ""
    for i, result in enumerate(search_response['vectors']):
        metadata = result['metadata']
        
        # Generate presigned URL for image
        bucket = metadata['img_path'].split('/')[2]
        key = '/'.join(metadata['img_path'].split('/')[3:])
        img_url = boto3.client('s3').generate_presigned_url(
            'get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600
        )
        
        html_content += f"""
        <div style="border: 1px solid #ccc; margin: 10px; padding: 10px; display: flex; align-items: center;">
            <img src="{img_url}" style="width: 150px; height: 150px; object-fit: contain; margin-right: 20px;">
            <div>
                <h3>{i+1}. {metadata['item_name']}</h3>
                <p><strong>Type:</strong> {metadata['type']} | <strong>Distance:</strong> {result['distance']:.3f}</p>
                <p><strong>Item ID:</strong> {metadata['item_id']}</p>
            </div>
        </div>
        """
    
    display(HTML(html_content))

### a. Query with Image

In [ ]:
# Test image query - use an image from the dataset
test_image_path = sample_dataset.iloc[0]['img_full_path']

# Display query image
from IPython.display import display, HTML
bucket = test_image_path.split('/')[2]
key = '/'.join(test_image_path.split('/')[3:])
query_img_url = boto3.client('s3').generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600)
display(HTML(f'<h3>Query Image:</h3><img src="{query_img_url}" style="width: 200px; height: 200px; object-fit: contain; border: 2px solid #007acc;">'))

query_embed = generate_image_embedding(test_image_path)
results = search_products(query_embed)
print("\nImage Search Results:")
display_results(results)

### b. Query with Text Only

In [ ]:
# Test text query
query_text = "15-ounce drinkware"
print(f"Searching for: {query_text}")

query_embed = generate_text_embedding(query_text, "IMAGE_RETRIEVAL")
results = search_products(query_embed)
print("\nText Search Results:")
display_results(results)

### c. Query with Hybrid Image and Text

In [ ]:
# Test hybrid query
query_text = "AmazonBasics drinkware set"
test_image_path = dataset.iloc[101]['img_full_path']  # Different image

# Display query image
bucket = test_image_path.split('/')[2]
key = '/'.join(test_image_path.split('/')[3:])
query_img_url = boto3.client('s3').generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600)
display(HTML(f'<h3>Hybrid Query - Text: "{query_text}"</h3><img src="{query_img_url}" style="width: 200px; height: 200px; object-fit: contain; border: 2px solid #007acc;">'))

text_embed = generate_text_embedding(query_text, "GENERIC_RETRIEVAL")
img_embed = generate_image_embedding(test_image_path, "GENERIC_RETRIEVAL")

# Combine embeddings (average)
hybrid_embed = [(t + i) / 2 for t, i in zip(text_embed, img_embed)]

results = search_products(hybrid_embed)
print("\nHybrid Search Results:")
display_results(results)